# Using Exotic Map Projections: `useCRS="provided"`

Value "provided" tells Lets-Plot that the input `SpatialDataset` already contains coordinates in the desired CRS and should not be reprojected any further.

This is a workaround for the case when the desired CRS is not naturally supported by Lets-Plot.

> Note: at the moment, Lets-Plot only supports **WGS84** CRS and the Mercator projection.

For example, if you try to plot a `SpatialDataset` which uses the **Conus Albers** CRS (EPSG:5070), then Lets-Plot will return an error:

> Geometry must use WGS84 coordinate reference system but was: NAD83 / Conus Albers.

Let's see how can we still plot a map in the **Conus Albers** projection in Lets-Plot.


In [1]:
%useLatestDescriptors
%use lets-plot

In [2]:
%use lets-plot-gt

In [3]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.1.2-alpha1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.3.0.0.

In [4]:
LetsPlot.theme = themeBW()

#### 1. Load North America `FeatureCollection` from "natural earth" Shapefiles


In [5]:
@file:DependsOn("org.geotools:gt-shapefile:[23,)")
@file:DependsOn("org.geotools:gt-cql:[23,)")
@file:DependsOn("org.geotools:gt-epsg-hsql:[23,)")


In [6]:
import org.geotools.data.shapefile.ShapefileDataStoreFactory
import org.geotools.data.simple.SimpleFeatureCollection
import org.geotools.filter.text.cql2.CQL
import java.net.URL

val factory = ShapefileDataStoreFactory()

In [7]:
val worldFeatures : SimpleFeatureCollection = with("naturalearth_lowres") {
    val url = "https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/shp/${this}/${this}.shp"
    factory.createDataStore(URL(url)).featureSource.features
}


In [8]:
worldFeatures.toSpatialDataset()["continent"]?.distinct()

[Oceania, Africa, North America, Asia, South America, Europe, Seven seas (open ocean), Antarctica]

In [9]:
// Obtain North America feature collection.
val northAmFc = worldFeatures.subCollection(
    CQL.toFilter("continent = 'North America'")
)

In [10]:
// Convert "North America" Feature Collection to a SpatialDataset.
// Use 10 decimals to encode floating point numbers (this is the default).
val northAmSds = northAmFc.toSpatialDataset(10)
northAmSds.crs

GCS_WGS_1984

#### 2. A Map of North America in the *Mercator* Projection

The Mercator projection is a map projection used by Lets-Plot by default when plotting a `SpatialDataset`.

Note that `northAmSds` dataset uses **WGS84** (or *GCS_WGS_1984*) CRS which is naitively supported by Lets-Plot.

In [11]:
letsPlot() + geomMap(map = northAmSds, fill = "blue", color = "white", size = 1)

#### 3. Reproject North America Feature Collection Using the *Conus Albers* Projection


In [12]:
import org.geotools.referencing.CRS
import org.geotools.data.store.ReprojectingFeatureCollection

In [13]:
val сonusAlbers = CRS.decode("EPSG:5070", true)

val northAmAlbersFc = ReprojectingFeatureCollection(northAmFc, сonusAlbers)

In [14]:
val northAmAlbersSds = northAmAlbersFc.toSpatialDataset(10)
northAmAlbersSds.crs

NAD83 / Conus Albers

#### 4. A Map of North America in the *Conus Albers* Projection


##### 4.1. Lets-Plot doesn't Support the *Conus Albers* Projection

In [15]:
// This attempt to create a map will fail with the error shown below.

// letsPlot() + geomMap(map = northAmAlbertsSds)

val p = letsPlot() + geomMap(map = northAmAlbersSds)

try {
    p.toSpec()
} catch(e:Exception) {
    println(e)
}

java.lang.IllegalArgumentException: Geometry must use WGS84 coordinate reference system but was: NAD83 / Conus Albers.


##### 4.2. A Map in the Conus Albers Projection with the Help of the `useCRS = "provided"` parameter

In [16]:
letsPlot() + 
    geomMap(map = northAmAlbersSds, useCRS = "provided", fill = "blue", color = "white", size = 1) +
    theme(axis = "blank")